#Archivo test

Se realizó un procedimiento similar al procesamiento de datos del archivo train, esto con el fin de que ambos archivos tengan la misma estructura.

In [ ]:
!pip install opendatasets
import opendatasets as od
dataset_link="https://www.kaggle.com/competitions/udea-ai4eng-20242/overview"
od.download(dataset_link)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: andresh7
Your Kaggle Key: ··········


100%|██████████| 20.1M/20.1M [00:00<00:00, 93.1MB/s]



Extracting archive ./udea-ai4eng-20242/udea-ai4eng-20242.zip to ./udea-ai4eng-20242


In [ ]:
import os
os.chdir("udea-ai4eng-20242")
os.listdir()

['submission_example.csv', 'test.csv', 'train.csv']

In [ ]:
import pandas as pd
df_train=pd.read_csv("train.csv")
df_test=pd.read_csv("test.csv")

In [ ]:
df_test.head()

,Unnamed: 0,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO
0,0,550236,20183,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Primaria completa,Si
1,1,98545,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Técnica o tecnológica completa,No
2,2,499179,20212,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Secundaria (Bachillerato) completa,No
3,3,782980,20195,CONTADURIA PUBLICA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Primaria incompleta,No
4,4,785185,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No


In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296786 entries, 0 to 296785
Data columns (total 12 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   Unnamed: 0                      296786 non-null  int64 
 1   ID                              296786 non-null  int64 
 2   PERIODO                         296786 non-null  int64 
 3   ESTU_PRGM_ACADEMICO             296786 non-null  object
 4   ESTU_PRGM_DEPARTAMENTO          296786 non-null  object
 5   ESTU_VALORMATRICULAUNIVERSIDAD  294063 non-null  object
 6   ESTU_HORASSEMANATRABAJA         283407 non-null  object
 7   FAMI_ESTRATOVIVIENDA            282991 non-null  object
 8   FAMI_TIENEINTERNET              285247 non-null  object
 9   FAMI_EDUCACIONPADRE             286793 non-null  object
 10  FAMI_EDUCACIONMADRE             286563 non-null  object
 11  ESTU_PAGOMATRICULAPROPIO        293979 non-null  object
dtypes: int64(3), object(9)
memory 

In [ ]:
k = df_test.isna().sum()
k[k!=0]


,0
ESTU_VALORMATRICULAUNIVERSIDAD,2723
ESTU_HORASSEMANATRABAJA,13379
FAMI_ESTRATOVIVIENDA,13795
FAMI_TIENEINTERNET,11539
FAMI_EDUCACIONPADRE,9993
FAMI_EDUCACIONMADRE,10223
ESTU_PAGOMATRICULAPROPIO,2807


In [ ]:
df_test['PERIODO'].unique()

array([20183, 20203, 20212, 20195, 20196, 20194, 20213, 20202, 20184])

In [ ]:
df_test.PERIODO.value_counts()

,count
PERIODO,
20195,77579
20203,73918
20212,73415
20183,70406
20194,604
20213,492
20202,190
20184,119
20196,63


In [ ]:
df_test['PERIODO'].isna().sum()

0

In [ ]:
df_test['PERIODO'] = df_test['PERIODO'].astype('category')

In [ ]:
df_test['PERIODO'] = df_test['PERIODO'].astype(str).str.replace(r'2018.*', '2018', regex=True)
df_test['PERIODO'] = df_test['PERIODO'].astype(str).str.replace(r'2019.*', '2019', regex=True)
df_test['PERIODO'] = df_test['PERIODO'].astype(str).str.replace(r'2020.*', '2020', regex=True)
df_test['PERIODO'] = df_test['PERIODO'].astype(str).str.replace(r'2021.*', '2021', regex=True)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encode = OneHotEncoder()


codificador = encode.fit_transform(df_test[['PERIODO']])

arreglo = codificador.toarray()
periodo = pd.DataFrame(arreglo,
                       columns = encode.categories_)


df_test = pd.concat([df_test, periodo], axis = 'columns')
df_test = df_test.drop('PERIODO', axis=1)

In [ ]:
df_test["ESTU_PRGM_ACADEMICO"].value_counts()

In [ ]:
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.6 MB/s eta 0:00:00


In [ ]:
from unidecode import unidecode
import re

replacements = {
    r'INGENIER\?A': 'INGENIERIA',
    r'INGENIIERIA': 'INGENIERIA',
    r'EDUCACI\?N\b': 'EDUCACION',
    r'RECREACI\?N\b': 'RECREACION',
    r'F\?SICA\b': 'FISICA',
    r'ESC\?NICAS\b': 'ESCENICAS',
    r'MATEM\?TICAS\b': 'MATEMATICAS',
    r'FILOSOF\?A\b': 'FILOSOFIA',
    r'DISE\?O\b': 'DISEÑO',
    r'DISE\?\?O\b': 'DISEÑO',
    r'COMUNICACI\?N\b': 'COMUNICACION',
    r'COMUNICACI\?\?N\b': 'COMUNICACION',
    r'GR\?\?FICA\b': 'GRAFICA',
    r'EL\?CTRICA\b': 'ELECTRICA',
    r'QU\?MICA\b': 'QUIMICA',
    r'FARMAC\?UTICA\b': 'FARMACEUTICA',
    r'ADMINISTRACI\?N\b': 'ADMINISTRACION',
    r'P\?BLICA\b': 'PUBLICA',
    r'INGL\?S\b': 'INGLES',
    r'\?NFASIS\b': 'ENFASIS',
    r'GESTI\?N\b': 'GESTION',
    r'M\?SICA\b': 'MUSICA',
    r'ECONOM\?A\b': 'ECONOMIA',
    r'GASTRONOM\?A\b': 'GASTRONOMIA',
    r'B\?SICA\b': 'BASICA',
    r'GEOLOG\?A\b': 'GEOLOGIA',
    r'PEDAGOG\?A\b': 'PEDAGOGIA',
    r'ART\?STICA\b': 'ARTISTICA',
    r'LOG\?STICA\b': 'LOGISTICA'
}

general_replacements = [
    (r'.*INGENIERIA.*', 'INGENIERIA'),
    (r'ADMINISTRACION.*', 'ADMINISTRACION'),
    (r'DISEÑO.*', 'DISEÑO'),
    (r'DISENO.*', 'DISEÑO'),
    (r'LICENCIATURA.*', 'LICENCIATURA'),
    (r'CIENCIA.*', 'CIENCIAS'),
    (r'COMUNICACION.*', 'COMUNICACION'),
    (r'GESTION.*', 'GESTION'),
    (r'FILOSOFIA.*', 'FILOSOFIA'),
    (r'QUIMICA.*', 'QUIMICA'),
    (r'NEGOCIOS.*', 'NEGOCIOS'),
    (r'.*MERCAD.*', 'MERCADEO'),
    (r'MATEMATICAS.*', 'MATEMATICAS'),
    (r'.*ARTE.*', 'ARTES'),
    (r'ESTUDIOS.*', 'ESTUDIOS'),
    (r'FINANZAS.*', 'FINANZAS'),
    (r'CONSTRUCCION.*', 'CONSTRUCCION'),
    (r'DIRECCION.*', 'DIRECCION'),
    (r'RELACIONES.*', 'RELACIONES'),
    (r'GEO.*', 'GEOGRAFIA'),
    (r'PROFESIONAL.*', 'PROFESIONAL'),
    (r'LENG.*', 'LENGUAS'),
    (r'ADMIN.*', 'ADMINISTRACION'),
    (r'.*DEPORT.*', 'DEPORTE'),
    (r'.*BIOLOGIA.*', 'BIOLOGIA'),
    (r'CINE.*', 'CINE'),
    (r'CONTADURIA.*', 'CONTADURIA'),
    (r'COMERCIO.*', 'COMERCIO'),
    (r'.*LITERA.*', 'LITERATURA'),
    (r'.*FILOL.*', 'FILOLOGIA'),
    (r'PUBLICI.*', 'PUBLICIDAD'),
    (r'MARK.*', 'MERCADEO'),
    (r'HISTORIA.*', 'HISTORIA'),
    (r'.*MUSICA.*', 'MUSICA'),
    (r'.*VETE.*', 'VETERINARIA'),
    (r'.*ZOO.*', 'VETERINARIA'),
    (r'.*GASTR.*', 'GASTRONOMIA'),
    (r'.*QUIRURGICA.*', 'MEDICINA'),
    (r'ECONO.*', 'ECONOMIA'),
    (r'GEREN.*', 'GERENCIA'),
    (r'HOTELERIA.*', 'HOTELERIA'),
    (r'.*PSICOLO.*', 'PSICOLOGIA'),
    (r'.*AGRO.*', 'AGRONOMIA')
]

def clean_text(text):
    text = unidecode(text).upper().strip()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'^PROFESIONAL EN\s+', '', text)
    for pattern, replacement in replacements.items():
        text = re.sub(pattern, replacement, text)
    for pattern, replacement in general_replacements:
        text = re.sub(pattern, replacement, text)
    return text

df_test['ESTU_PRGM_ACADEMICO'] = df_test['ESTU_PRGM_ACADEMICO'].apply(clean_text)

In [ ]:
len(df_test['ESTU_PRGM_ACADEMICO'].unique())

109

In [ ]:
programas_agrupados=['INGENIERIA','ADMINISTRACION','DISEÑO','LICENCIATURA','CIENCIAS','COMUNICACION','GESTION','FILOSOFIA','QUIMICA','NEGOCIOS','MERCADEO',
'MATEMATICAS','ARTES','ESTUDIOS','FINANZAS','CONSTRUCCION','DIRECCION','RELACIONES','GEOGRAFIA','PROFESIONAL','LENGUAS', 'DEPORTE'
'BIOLOGIA','CINE','CONTADURIA','COMERCIO','LITERATURA','FILOLOGIA','PUBLICIDAD','HISTORIA','MUSICA','VETERINARIA','GASTRONOMIA'
'MEDICINA', 'ECONOMIA','GERENCIA','HOTELERIA','PSICOLOGIA','AGRONOMIA']

programas_otros = [elem for elem in df_test['ESTU_PRGM_ACADEMICO'] if elem not in programas_agrupados]

condicion = df_test['ESTU_PRGM_ACADEMICO'].isin(programas_otros)
df_test.loc[condicion, 'ESTU_PRGM_ACADEMICO'] = 'OTRAS'

In [ ]:
df_test['ESTU_PRGM_ACADEMICO'].unique()

array(['OTRAS', 'ADMINISTRACION', 'INGENIERIA', 'CONTADURIA', 'ECONOMIA',
       'PSICOLOGIA', 'NEGOCIOS', 'VETERINARIA', 'LICENCIATURA', 'GESTION',
       'FILOSOFIA', 'COMUNICACION', 'FINANZAS', 'CIENCIAS', 'DISEÑO',
       'GEOGRAFIA', 'MERCADEO', 'MUSICA', 'COMERCIO', 'CINE',
       'PUBLICIDAD', 'LENGUAS', 'RELACIONES', 'LITERATURA', 'ARTES',
       'MATEMATICAS', 'HOTELERIA', 'QUIMICA', 'AGRONOMIA', 'ESTUDIOS',
       'CONSTRUCCION', 'GERENCIA', 'PROFESIONAL', 'DIRECCION', 'HISTORIA',
       'FILOLOGIA'], dtype=object)

In [ ]:
encode = OneHotEncoder()
codificadorPr = encode.fit_transform(df_test[['ESTU_PRGM_ACADEMICO']])
arregloPr = codificadorPr.toarray()
programa = pd.DataFrame(arregloPr,columns = encode.categories_)

In [ ]:
df_test = pd.concat([df_test, programa], axis = 'columns')
df_test = df_test.drop('ESTU_PRGM_ACADEMICO', axis=1)

In [ ]:
df_test.head()

,Unnamed: 0,ID,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,...,"(MERCADEO,)","(MUSICA,)","(NEGOCIOS,)","(OTRAS,)","(PROFESIONAL,)","(PSICOLOGIA,)","(PUBLICIDAD,)","(QUIMICA,)","(RELACIONES,)","(VETERINARIA,)"
0,0,550236,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Primaria completa,Si,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,98545,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Técnica o tecnológica completa,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,499179,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Secundaria (Bachillerato) completa,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,782980,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Primaria incompleta,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,785185,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_test['ESTU_PRGM_DEPARTAMENTO'].unique()

array(['BOLIVAR', 'ANTIOQUIA', 'BOGOTÁ', 'SUCRE', 'ATLANTICO', 'VALLE',
       'SANTANDER', 'RISARALDA', 'QUINDIO', 'CASANARE', 'NORTE SANTANDER',
       'CAUCA', 'HUILA', 'CALDAS', 'CHOCO', 'CORDOBA', 'LA GUAJIRA',
       'META', 'NARIÑO', 'BOYACA', 'TOLIMA', 'CUNDINAMARCA', 'MAGDALENA',
       'CESAR', 'PUTUMAYO', 'CAQUETA', 'ARAUCA', 'AMAZONAS', 'VAUPES',
       'SAN ANDRES', 'GUAVIARE'], dtype=object)

In [ ]:
#Agrupar region andina
def region_andina(dataframe, columna):
    reemplazos_andina = {
        'ANTIOQUIA': 'ANDINA',
        'BOYACA': 'ANDINA',
        'BOGOTÁ': 'ANDINA',
        'CALDAS': 'ANDINA',
        'CUNDINAMARCA': 'ANDINA',
        'HUILA': 'ANDINA',
        'NORTE SANTANDER': 'ANDINA',
        'QUINDIO': 'ANDINA',
        'RISARALDA': 'ANDINA',
        'SANTANDER': 'ANDINA',
        'TOLIMA': 'ANDINA'
    }
    for valor_original, valor_reemplazo in reemplazos_andina.items():
        dataframe[columna] = dataframe[columna].astype(str).str.replace(valor_original, valor_reemplazo, regex=True)
    return dataframe

#Agrupar region amazonia
def region_amazonia(dataframe, columna):
    reemplazos_amazonia = {
        'AMAZONAS': 'AMAZONIA',
        'CAQUETA': 'AMAZONIA',
        'VAUPES': 'AMAZONIA',
        'GUAINIA': 'AMAZONIA',
        'GUAVIARE': 'AMAZONIA',
        'PUTUMAYO': 'AMAZONIA'
    }
    for valor_original, valor_reemplazo in reemplazos_amazonia.items():
        dataframe[columna] = dataframe[columna].astype(str).str.replace(valor_original, valor_reemplazo, regex=True)
    return dataframe

#Agrupar region pacifica
def region_pacifica(dataframe, columna):
    reemplazos_pacifica = {
        'VALLE': 'PACIFICA',
        'CHOCO': 'PACIFICA',
        'CAUCA': 'PACIFICA',
        'NARIÑO': 'PACIFICA',
    }
    for valor_original, valor_reemplazo in reemplazos_pacifica.items():
        dataframe[columna] = dataframe[columna].astype(str).str.replace(valor_original, valor_reemplazo, regex=True)
    return dataframe

#Agrupar region caribe
def region_caribe(dataframe, columna):
    reemplazos_caribe = {
        'ATLANTICO': 'CARIBE',
        'BOLIVAR': 'CARIBE',
        'CESAR': 'CARIBE',
        'CORDOBA': 'CARIBE',
        'LA GUAJIRA': 'CARIBE',
        'MAGDALENA': 'CARIBE',
        'SUCRE': 'CARIBE',
        'SAN ANDRES': 'CARIBE'
    }
    for valor_original, valor_reemplazo in reemplazos_caribe.items():
        dataframe[columna] = dataframe[columna].astype(str).str.replace(valor_original, valor_reemplazo, regex=True)
    return dataframe

#Agrupar region orinoquia
def region_orinoquia(dataframe, columna):
    reemplazos_orinoquia = {
        'ARAUCA': 'ORINOQUIA',
        'CASANARE': 'ORINOQUIA',
        'META': 'ORINOQUIA',
        'VICHADA': 'ORINOQUIA'
    }
    for valor_original, valor_reemplazo in reemplazos_orinoquia.items():
        dataframe[columna] = dataframe[columna].astype(str).str.replace(valor_original, valor_reemplazo, regex=True)
    return dataframe

In [ ]:
df_test = region_andina(df_test, 'ESTU_PRGM_DEPARTAMENTO')
df_test = region_amazonia(df_test, 'ESTU_PRGM_DEPARTAMENTO')
df_test = region_pacifica(df_test, 'ESTU_PRGM_DEPARTAMENTO')
df_test = region_caribe(df_test, 'ESTU_PRGM_DEPARTAMENTO')
df_test = region_orinoquia(df_test, 'ESTU_PRGM_DEPARTAMENTO')

In [ ]:
print(df_test['ESTU_PRGM_DEPARTAMENTO'].unique())
print(df_test['ESTU_PRGM_DEPARTAMENTO'].isna().sum())

['CARIBE' 'ANDINA' 'PACIFICA' 'ORINOQUIA' 'AMAZONIA']
0


In [ ]:
encode = OneHotEncoder()
codificadorDepto = encode.fit_transform(df_test[['ESTU_PRGM_DEPARTAMENTO']])

arrayDepartamento = codificadorDepto.toarray()
departamento = pd.DataFrame(arrayDepartamento,
                       columns = encode.categories_)
df_test = pd.concat([df_test, departamento], axis = 'columns')
df_test = df_test.drop('ESTU_PRGM_DEPARTAMENTO', axis=1)

In [ ]:
df_test.head()

,Unnamed: 0,ID,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,"(2018,)",...,"(PSICOLOGIA,)","(PUBLICIDAD,)","(QUIMICA,)","(RELACIONES,)","(VETERINARIA,)","(AMAZONIA,)","(ANDINA,)","(CARIBE,)","(ORINOQUIA,)","(PACIFICA,)"
0,0,550236,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Primaria completa,Si,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1,98545,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Técnica o tecnológica completa,No,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2,499179,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Secundaria (Bachillerato) completa,No,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,3,782980,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Primaria incompleta,No,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,4,785185,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
df_test['ESTU_VALORMATRICULAUNIVERSIDAD'].unique()

array(['Menos de 500 mil', 'Entre 2.5 millones y menos de 4 millones',
       'Entre 1 millón y menos de 2.5 millones',
       'Entre 500 mil y menos de 1 millón', 'Más de 7 millones',
       'Entre 4 millones y menos de 5.5 millones',
       'Entre 5.5 millones y menos de 7 millones', 'No pagó matrícula',
       nan], dtype=object)

In [ ]:
df_test['ESTU_VALORMATRICULAUNIVERSIDAD'].isna().sum()

2723

In [ ]:
df_test['ESTU_VALORMATRICULAUNIVERSIDAD'] = df_test['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna('No pagó matrícula')

In [ ]:
df_test['ESTU_VALORMATRICULAUNIVERSIDAD'].isna().sum()

0

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
valores_matricula = ['No pagó matrícula','Menos de 500 mil','Entre 500 mil y menos de 1 millón',
                     'Entre 1 millón y menos de 2.5 millones',
                     'Entre 2.5 millones y menos de 4 millones',
                     'Entre 4 millones y menos de 5.5 millones',
                     'Entre 5.5 millones y menos de 7 millones','Más de 7 millones']

encode = OneHotEncoder()

codificadorPr = encode.fit_transform(df_test[['ESTU_VALORMATRICULAUNIVERSIDAD']])
arregloPr = codificadorPr.toarray()
matricula = pd.DataFrame(arregloPr,
                       columns = encode.categories_)


In [ ]:
df_test = pd.concat([df_test, matricula], axis = 'columns')
df_test = df_test.drop('ESTU_VALORMATRICULAUNIVERSIDAD',axis=1)

In [ ]:
df_test.head()

,Unnamed: 0,ID,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,"(2018,)","(2019,)",...,"(ORINOQUIA,)","(PACIFICA,)","(Entre 1 millón y menos de 2.5 millones,)","(Entre 2.5 millones y menos de 4 millones,)","(Entre 4 millones y menos de 5.5 millones,)","(Entre 5.5 millones y menos de 7 millones,)","(Entre 500 mil y menos de 1 millón,)","(Menos de 500 mil,)","(Más de 7 millones,)","(No pagó matrícula,)"
0,0,550236,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Primaria completa,Si,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1,98545,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Técnica o tecnológica completa,No,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,499179,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Secundaria (Bachillerato) completa,No,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,782980,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Primaria incompleta,No,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,785185,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_test['ESTU_HORASSEMANATRABAJA'].unique()

array(['Menos de 10 horas', 'Entre 21 y 30 horas', '0',
       'Entre 11 y 20 horas', 'Más de 30 horas', nan], dtype=object)

In [ ]:
df_test['ESTU_HORASSEMANATRABAJA'].value_counts()

,count
ESTU_HORASSEMANATRABAJA,
Más de 30 horas,106189
0,50422
Entre 11 y 20 horas,49386
Entre 21 y 30 horas,39931
Menos de 10 horas,37479


In [ ]:
df_test['ESTU_HORASSEMANATRABAJA'].isna().sum()

13379

In [ ]:
df_test['ESTU_HORASSEMANATRABAJA'] = df_test['ESTU_HORASSEMANATRABAJA'].fillna('0')

In [ ]:
df_test['ESTU_HORASSEMANATRABAJA'].isna().sum()

0

In [ ]:
horas_trabajo = ['0','Menos de 10 horas','Entre 11 y 20 horas',
                'Entre 21 y 30 horas',
                'Más de 30 horas']
encode = OneHotEncoder()

#Trabaja con dataframes, no series
codificadorPr = encode.fit_transform(df_test[['ESTU_HORASSEMANATRABAJA']])
arregloPr = codificadorPr.toarray()
horas_trabajo = pd.DataFrame(arregloPr,
                       columns = encode.categories_)
df_test = pd.concat([df_test, horas_trabajo], axis = 'columns')
df_test = df_test.drop('ESTU_HORASSEMANATRABAJA',axis=1)

In [ ]:
df_test.head()

,Unnamed: 0,ID,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,"(2018,)","(2019,)","(2020,)",...,"(Entre 5.5 millones y menos de 7 millones,)","(Entre 500 mil y menos de 1 millón,)","(Menos de 500 mil,)","(Más de 7 millones,)","(No pagó matrícula,)","(0,)","(Entre 11 y 20 horas,)","(Entre 21 y 30 horas,)","(Menos de 10 horas,)","(Más de 30 horas,)"
0,0,550236,Estrato 3,Si,Técnica o tecnológica completa,Primaria completa,Si,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,98545,Estrato 2,Si,Secundaria (Bachillerato) completa,Técnica o tecnológica completa,No,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2,499179,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Secundaria (Bachillerato) completa,No,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,3,782980,Estrato 1,No,Primaria incompleta,Primaria incompleta,No,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,4,785185,Estrato 2,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
df_test['FAMI_ESTRATOVIVIENDA'].unique()

array(['Estrato 3', 'Estrato 2', 'Estrato 1', 'Estrato 4', nan,
       'Estrato 6', 'Estrato 5', 'Sin Estrato'], dtype=object)

In [ ]:
df_test['FAMI_ESTRATOVIVIENDA'].value_counts()

,count
FAMI_ESTRATOVIVIENDA,
Estrato 2,99288
Estrato 3,90719
Estrato 1,48092
Estrato 4,27823
Estrato 5,10194
Estrato 6,5451
Sin Estrato,1424


In [ ]:
df_test['FAMI_ESTRATOVIVIENDA'].isna().sum()

13795

In [ ]:
df_test['FAMI_ESTRATOVIVIENDA'] = df_test['FAMI_ESTRATOVIVIENDA'].fillna(df_test['FAMI_ESTRATOVIVIENDA'].mode()[0])

In [ ]:
df_test['FAMI_ESTRATOVIVIENDA'].isna().sum()

0

In [ ]:
estratos = ['Sin Estrato','Estrato 1','Estrato 2',
            'Estrato 3', 'Estrato 4', 'Estrato 5',
            'Estrato 6']
encode = OneHotEncoder()

#Trabaja con dataframes, no series
codificadorPr = encode.fit_transform(df_test[['FAMI_ESTRATOVIVIENDA']])
arregloPr = codificadorPr.toarray()
estrato = pd.DataFrame(arregloPr,
                       columns = encode.categories_)
df_test = pd.concat([df_test, estrato], axis = 'columns')
df_test = df_test.drop('FAMI_ESTRATOVIVIENDA',axis=1)

In [ ]:
df_test.head()

,Unnamed: 0,ID,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,"(2018,)","(2019,)","(2020,)","(2021,)",...,"(Entre 21 y 30 horas,)","(Menos de 10 horas,)","(Más de 30 horas,)","(Estrato 1,)","(Estrato 2,)","(Estrato 3,)","(Estrato 4,)","(Estrato 5,)","(Estrato 6,)","(Sin Estrato,)"
0,0,550236,Si,Técnica o tecnológica completa,Primaria completa,Si,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,98545,Si,Secundaria (Bachillerato) completa,Técnica o tecnológica completa,No,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2,499179,Si,Secundaria (Bachillerato) incompleta,Secundaria (Bachillerato) completa,No,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,3,782980,No,Primaria incompleta,Primaria incompleta,No,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,785185,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_test['FAMI_TIENEINTERNET'].unique()

array(['Si', 'No', nan], dtype=object)

In [ ]:
df_test['FAMI_TIENEINTERNET'].isna().sum()

11539

In [ ]:
df_test['FAMI_TIENEINTERNET'].value_counts()

,count
FAMI_TIENEINTERNET,
Si,254075
No,31172


In [ ]:
df_test['FAMI_TIENEINTERNET'] = df_test['FAMI_TIENEINTERNET'].fillna("No")

In [ ]:
df_test['FAMI_TIENEINTERNET'].isna().sum()

0

In [ ]:
df_test['FAMI_TIENEINTERNET'] = df_test['FAMI_TIENEINTERNET'].replace(r'Si', 1.).infer_objects()
df_test['FAMI_TIENEINTERNET'] = df_test['FAMI_TIENEINTERNET'].replace(r'No', 0.).infer_objects()

<ipython-input-63-4e6829c1e75b>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_test['FAMI_TIENEINTERNET'] = df_test['FAMI_TIENEINTERNET'].replace(r'No', 0.).infer_objects()


In [ ]:
df_test['FAMI_TIENEINTERNET'].value_counts()

,count
FAMI_TIENEINTERNET,
1.0,254075
0.0,42711


In [ ]:
df_test.head()

,Unnamed: 0,ID,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,"(2018,)","(2019,)","(2020,)","(2021,)",...,"(Entre 21 y 30 horas,)","(Menos de 10 horas,)","(Más de 30 horas,)","(Estrato 1,)","(Estrato 2,)","(Estrato 3,)","(Estrato 4,)","(Estrato 5,)","(Estrato 6,)","(Sin Estrato,)"
0,0,550236,1.0,Técnica o tecnológica completa,Primaria completa,Si,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,98545,1.0,Secundaria (Bachillerato) completa,Técnica o tecnológica completa,No,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2,499179,1.0,Secundaria (Bachillerato) incompleta,Secundaria (Bachillerato) completa,No,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,3,782980,0.0,Primaria incompleta,Primaria incompleta,No,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,785185,1.0,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_test['ESTU_PAGOMATRICULAPROPIO'].unique()

array(['Si', 'No', nan], dtype=object)

In [ ]:
df_test['ESTU_PAGOMATRICULAPROPIO'].value_counts()

,count
ESTU_PAGOMATRICULAPROPIO,
No,163869
Si,130110


In [ ]:
df_test['ESTU_PAGOMATRICULAPROPIO'].isna().sum()

2807

In [ ]:
df_test['ESTU_PAGOMATRICULAPROPIO'] = df_test['ESTU_PAGOMATRICULAPROPIO'].fillna(df_test['ESTU_PAGOMATRICULAPROPIO'].mode()[0])

In [ ]:
df_test['ESTU_PAGOMATRICULAPROPIO'].isna().sum()

0

In [ ]:
df_test['ESTU_PAGOMATRICULAPROPIO'] = df_test['ESTU_PAGOMATRICULAPROPIO'].replace(r'Si', 1.).infer_objects()
df_test['ESTU_PAGOMATRICULAPROPIO'] = df_test['ESTU_PAGOMATRICULAPROPIO'].replace(r'No', 0.).infer_objects()

<ipython-input-71-39dce4344262>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_test['ESTU_PAGOMATRICULAPROPIO'] = df_test['ESTU_PAGOMATRICULAPROPIO'].replace(r'No', 0.).infer_objects()


In [ ]:
df_test['ESTU_PAGOMATRICULAPROPIO'].value_counts()

,count
ESTU_PAGOMATRICULAPROPIO,
0.0,166676
1.0,130110


In [ ]:
df_test.head()

,Unnamed: 0,ID,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,"(2018,)","(2019,)","(2020,)","(2021,)",...,"(Entre 21 y 30 horas,)","(Menos de 10 horas,)","(Más de 30 horas,)","(Estrato 1,)","(Estrato 2,)","(Estrato 3,)","(Estrato 4,)","(Estrato 5,)","(Estrato 6,)","(Sin Estrato,)"
0,0,550236,1.0,Técnica o tecnológica completa,Primaria completa,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,98545,1.0,Secundaria (Bachillerato) completa,Técnica o tecnológica completa,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2,499179,1.0,Secundaria (Bachillerato) incompleta,Secundaria (Bachillerato) completa,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,3,782980,0.0,Primaria incompleta,Primaria incompleta,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,785185,1.0,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_test['FAMI_EDUCACIONPADRE'].unique()

array(['Técnica o tecnológica completa',
       'Secundaria (Bachillerato) completa',
       'Secundaria (Bachillerato) incompleta', 'Primaria incompleta',
       'Ninguno', 'Primaria completa', 'Postgrado', nan,
       'Educación profesional completa',
       'Técnica o tecnológica incompleta',
       'Educación profesional incompleta', 'No sabe', 'No Aplica'],
      dtype=object)

In [ ]:
df_test['FAMI_EDUCACIONPADRE'].value_counts()

,count
FAMI_EDUCACIONPADRE,
Secundaria (Bachillerato) completa,54838
Primaria incompleta,53432
Educación profesional completa,35412
Secundaria (Bachillerato) incompleta,30641
Técnica o tecnológica completa,27045
Primaria completa,24131
Postgrado,19287
Educación profesional incompleta,11744
Técnica o tecnológica incompleta,9677


In [ ]:
df_test['FAMI_EDUCACIONPADRE'].isna().sum()

9993

In [ ]:
df_test['FAMI_EDUCACIONPADRE'] = df_test['FAMI_EDUCACIONPADRE'].replace(r'Ninguno', 'No Aplica').infer_objects()
df_test['FAMI_EDUCACIONPADRE'] = df_test['FAMI_EDUCACIONPADRE'].replace(r'No sabe', 'No Aplica').infer_objects()
df_test['FAMI_EDUCACIONPADRE'] = df_test['FAMI_EDUCACIONPADRE'].fillna('No Aplica')

In [ ]:
df_test['FAMI_EDUCACIONPADRE'].isna().sum()

0

In [ ]:
df_test['FAMI_EDUCACIONPADRE'].value_counts()

,count
FAMI_EDUCACIONPADRE,
Secundaria (Bachillerato) completa,54838
Primaria incompleta,53432
Educación profesional completa,35412
Secundaria (Bachillerato) incompleta,30641
No Aplica,30579
Técnica o tecnológica completa,27045
Primaria completa,24131
Postgrado,19287
Educación profesional incompleta,11744


In [ ]:

codificadorPr = encode.fit_transform(df_test[['FAMI_EDUCACIONPADRE']])

arregloPr = codificadorPr.toarray()
edu_padre = pd.DataFrame(arregloPr,
                       columns = encode.categories_)
df_test = pd.concat([df_test, edu_padre], axis = 'columns')
df_test = df_test.drop('FAMI_EDUCACIONPADRE',axis=1)

In [ ]:
df_test.head()

,Unnamed: 0,ID,FAMI_TIENEINTERNET,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,"(2018,)","(2019,)","(2020,)","(2021,)","(ADMINISTRACION,)",...,"(Educación profesional completa,)","(Educación profesional incompleta,)","(No Aplica,)","(Postgrado,)","(Primaria completa,)","(Primaria incompleta,)","(Secundaria (Bachillerato) completa,)","(Secundaria (Bachillerato) incompleta,)","(Técnica o tecnológica completa,)","(Técnica o tecnológica incompleta,)"
0,0,550236,1.0,Primaria completa,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,98545,1.0,Técnica o tecnológica completa,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2,499179,1.0,Secundaria (Bachillerato) completa,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,3,782980,0.0,Primaria incompleta,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,4,785185,1.0,Secundaria (Bachillerato) completa,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
df_test['FAMI_EDUCACIONMADRE'].unique()

array(['Primaria completa', 'Técnica o tecnológica completa',
       'Secundaria (Bachillerato) completa', 'Primaria incompleta',
       'Postgrado', 'Secundaria (Bachillerato) incompleta', nan,
       'Ninguno', 'Educación profesional completa',
       'Técnica o tecnológica incompleta',
       'Educación profesional incompleta', 'No sabe', 'No Aplica'],
      dtype=object)

In [ ]:
df_test['FAMI_EDUCACIONMADRE'].value_counts()

,count
FAMI_EDUCACIONMADRE,
Secundaria (Bachillerato) completa,60649
Primaria incompleta,42149
Técnica o tecnológica completa,38571
Educación profesional completa,36477
Secundaria (Bachillerato) incompleta,34566
Primaria completa,24399
Postgrado,20049
Técnica o tecnológica incompleta,11650
Educación profesional incompleta,9647


In [ ]:
df_test['FAMI_EDUCACIONMADRE'].isna().sum()

10223

In [ ]:
df_test['FAMI_EDUCACIONMADRE'] = df_test['FAMI_EDUCACIONMADRE'].replace(r'Ninguno', 'No Aplica').infer_objects()
df_test['FAMI_EDUCACIONMADRE'] = df_test['FAMI_EDUCACIONMADRE'].replace(r'No sabe', 'No Aplica').infer_objects()
df_test['FAMI_EDUCACIONMADRE'] = df_test['FAMI_EDUCACIONMADRE'].fillna('No Aplica')

In [ ]:
df_test['FAMI_EDUCACIONMADRE'].isna().sum()

0

In [ ]:

encode = OneHotEncoder()

codificadorPr = encode.fit_transform(df_test[['FAMI_EDUCACIONMADRE']])

arregloPr = codificadorPr.toarray()
edu_madre = pd.DataFrame(arregloPr,
                       columns = encode.categories_)
df_test = pd.concat([df_test, edu_madre], axis = 'columns')
df_test = df_test.drop('FAMI_EDUCACIONMADRE',axis=1)

In [ ]:
df_test.head()

,Unnamed: 0,ID,FAMI_TIENEINTERNET,ESTU_PAGOMATRICULAPROPIO,"(2018,)","(2019,)","(2020,)","(2021,)","(ADMINISTRACION,)","(AGRONOMIA,)",...,"(Educación profesional completa,)","(Educación profesional incompleta,)","(No Aplica,)","(Postgrado,)","(Primaria completa,)","(Primaria incompleta,)","(Secundaria (Bachillerato) completa,)","(Secundaria (Bachillerato) incompleta,)","(Técnica o tecnológica completa,)","(Técnica o tecnológica incompleta,)"
0,0,550236,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,98545,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2,499179,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,3,782980,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,4,785185,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
file_excel = df_train.to_csv("df_test_PPC2.csv", index = False)

In [ ]:
from google.colab import files
files.download('df_test_PPC2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>